In [12]:
#Lab_1 prepared by Bekzat Bakytbek - resubmission
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import pandas as pd
from nltk.stem import WordNetLemmatizer 
import nltk
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import re, string
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
from collections import defaultdict
import collections
d = collections.defaultdict(int)
from gensim.models import Word2Vec

In [13]:
#read the csv file using pandas
bbc_dataframe = pd.read_csv('bbc_text.csv')
#function returns first 20 elements of dataframe called bbc
bbc_dataframe.head(20)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...
7,sport,henman hopes ended in dubai third seed tim hen...
8,sport,wilkinson fit to face edinburgh england captai...
9,entertainment,last star wars not for children the sixth an...


In [14]:
#function that filters raw data (bbc data) into clean tokens removing punctuations, 
#numbers & converting sentences to lowercases
def clean_corpus(sentence):
    sentence = sentence.lower() #conveting to lowercase
    sentence = re.sub(r'\[.*?\]', '', sentence) #removing punctuations
    sentence = re.sub(r'[%s]' % re.escape(string.punctuation), '', sentence) #removing punctuations
    sentence = re.sub(r'\w*\d\w*', '', sentence) #removing punctuations
    if len(sentence) > 2:
        return ' '.join(w for w in sentence.split() if w not in STOPWORDS)

In [15]:
#source https://thispointer.com/pandas-apply-apply-a-function-to-each-row-column-in-dataframe/
filtered_corpus = pd.DataFrame(bbc_dataframe.text.apply(lambda x: clean_corpus(x)))
#print filtered corpus
filtered_corpus.head(20)

,text
0,tv future hands viewers home theatre systems p...
1,worldcom boss left books alone former worldcom...
2,tigers wary farrell gamble leicester say rushe...
3,yeading face newcastle fa cup premiership side...
4,ocean twelve raids box office ocean twelve cri...
5,howard hits back mongrel jibe michael howard s...
6,blair prepares name poll date tony blair likel...
7,henman hopes ended dubai third seed tim henman...
8,wilkinson fit face edinburgh england captain j...
9,last star wars children sixth final star wars ...


In [16]:
#function that applies lemmatizer using spacy
#source: https://spacy.io/usage/models
def lemmatizer(sentence):        
    lemma = []
    corpus_doc = nlp(sentence)
    for w in corpus_doc:
        lemma.append(w.lemma_)
    return lemma

#source https://thispointer.com/pandas-apply-apply-a-function-to-each-row-column-in-dataframe/
filtered_corpus["text"] =  filtered_corpus.apply(lambda x: lemmatizer(x['text']), axis=1)

In [17]:
filtered_corpus.head(20)

,text
0,"[tv, future, hand, viewer, home, theatre, syst..."
1,"[worldcom, boss, leave, book, alone, former, w..."
2,"[tigers, wary, farrell, gamble, leicester, say..."
3,"[yeade, face, newcastle, fa, cup, premiership,..."
4,"[ocean, twelve, raid, box, office, ocean, twel..."
5,"[howard, hit, back, mongrel, jibe, michael, ho..."
6,"[blair, prepare, name, poll, date, tony, blair..."
7,"[henman, hope, end, dubai, third, seed, tim, h..."
8,"[wilkinson, fit, face, edinburgh, england, cap..."
9,"[last, star, war, child, sixth, final, star, w..."


In [18]:
#source https://www.accelebrate.com/blog/using-defaultdict-python
for w in filtered_corpus['text']:
    for i in w:
        d[i] += 1
len(d)

25893

In [19]:
sorted(d, key=d.get, reverse=True)[:5]

['say', 'mr', 'year', 'would', 'make']

In [20]:
#source https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92
model = Word2Vec(size=1000, min_count=100, window=2, workers=3, iter = 10) #setting the paramethers
model.build_vocab(filtered_corpus['text']) #build our vocabulary
model.train(filtered_corpus['text'], total_examples=model.corpus_count, epochs=model.iter) #training our model
model.init_sims(replace = True)
model.save('word2vec_model')
model = Word2Vec.load('word2vec_model')
similarities = model.wv.most_similar('america')

for word , score in similarities:
    print(word , score)

C:\Users\Bekzat\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


north 0.8746350407600403
south 0.8526561260223389
east 0.8177163600921631
trade 0.7740395069122314
german 0.7724226117134094
australia 0.7637919187545776
annual 0.7452062964439392
africa 0.7439569234848022
germany 0.7369859218597412
exchange 0.7268362045288086
